## Create project

This notebook creates a project for OpenSfM to run a reconstruction on. Prerequisites:
- Create a directory called projects/ within the current directory
- Within projects/, create a folder with whatever name you want to give to your current reconstruction
- Within this individual project, download a geojson called map.geojson with your AOI from https://geojson.io/

In [1]:
# downloading the metadata file if it does not exist
import os
if not os.path.isfile("ladi_images_metadata.csv"):
    !wget http://ladi.s3-us-west-2.amazonaws.com/Labels/ladi_images_metadata.csv

--2020-08-19 14:28:54--  http://ladi.s3-us-west-2.amazonaws.com/Labels/ladi_images_metadata.csv
Resolving ladi.s3-us-west-2.amazonaws.com (ladi.s3-us-west-2.amazonaws.com)... 52.218.144.5
Connecting to ladi.s3-us-west-2.amazonaws.com (ladi.s3-us-west-2.amazonaws.com)|52.218.144.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 183993937 (175M) [text/csv]
Saving to: ‘ladi_images_metadata.csv’

ladi_images_metadat 100%[===================>] 175.47M   266KB/s    in 10m 6s  

2020-08-19 14:39:00 (297 KB/s) - ‘ladi_images_metadata.csv’ saved [183993937/183993937]



In [2]:
# importing the usual things
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import json
import shutil

# for downloading images
import requests

# importing the geoprocessing libraries
import geopandas as gpd
import fiona # library for reading/writing GIS files, comes w/ geopandas
from shapely.geometry import Point, LineString, Polygon

In [3]:
# change parameters for individual projects
target_date = "2017-10-13"
project_path = "projects/mercedita/"

In [6]:
# load the csv as a gdf
df = pd.read_csv("ladi_images_metadata.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index("timestamp", inplace=True)
df = df.loc[target_date]

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.gps_lon, df.gps_lat))
gdf.crs = 'epsg:4326'

geojson = gpd.read_file(project_path + 'map.geojson')
poly = geojson.iloc[0, 0]

In [7]:
# filter out images that are outside of the geojson
sample = gdf[gdf.geometry.apply(lambda x: x.within(poly))]

In [9]:
def create_project(sample, project_path):
    # copying the config file
    try:
        shutil.copy("config.yaml", project_path)
    except IOError as e:
        print("Unable to copy file. %s" % e)
    except:
        print("Unexpected error:", sys.exc_info())
    
    # get the list of urls to download
    url_list = sample["url"].to_list()
    
    # create the images folder if it does not currently exist
    image_folder_path = project_path + "images/"
    if not os.path.exists(image_folder_path):
        os.mkdir(image_folder_path)
        
    raise
    # iterate over the images, download them all
    for url in url_list:
        image_name = url.split("/")[-1]
        image_path = image_folder_path + image_name
        if os.path.isfile(image_path):
            continue
        try:
            r = requests.get(url)
        except:
            continue
        with open(image_path, 'wb') as f:
            f.write(r.content)

In [10]:
create_project(sample, project_path)

RuntimeError: No active exception to reraise